# Machine Learning 101++ in Python

#### Created by:

Pieter Buteneers ([@PieterButeneers](https://twitter.com/pieterbuteneers)), CTO at [chatlayer.ai](https://www.chatlayer.ai/) 

Bart De Vylder, Senior Research Engineer at [Google DeepMind](https://deepmind.com/)

Jeroen Boeye ([@JeroenBoeye](https://twitter.com/JeroenBoeye)), Senior Machine Learning Engineer at [Faktion](https://www.faktion.com/)

Joris Boeye ([@JorisBoeye](https://twitter.com/JorisBoeye)), Senior Data Scientist at [ZF Wind Power](https://www.zf.com/products/en/wind/home/wind.html)

### About

This tutorial consists of 3 parts, the first one will introduce you to Jupyter notebooks and Numpy to get you started with Python. The second part covers the basics of machine learning and the third shows how to build a recommendation engine. Finally, there is a bonus part on time series anomaly detection. Click on the links below to get started!


## [1. Jupyter notebooks and Numpy](notebook_intro.ipynb)

## [2. Machine Learning](machine_learning.ipynb)

## [3. Recommendation Engine](recommendation_engine.ipynb)

## Extras

[Simple Time Series Anomaly Detection](anomaly_detection.ipynb)